<a href="https://colab.research.google.com/github/RaIvPa/Improving-BERT-for-Biomedical-QA/blob/main/practice/huggingface_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>